# T1021.006 - Windows Remote Management
Adversaries may use [Valid Accounts](https://attack.mitre.org/techniques/T1078) to interact with remote systems using Windows Remote Management (WinRM). The adversary may then perform actions as the logged-on user.

WinRM is the name of both a Windows service and a protocol that allows a user to interact with a remote system (e.g., run an executable, modify the Registry, modify services).(Citation: Microsoft WinRM) It may be called with the `winrm` command or by any number of programs such as PowerShell.(Citation: Jacobsen 2014) WinRM  can be used as a method of remotely interacting with [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047).(Citation: MSDN WMI)

## Atomic Tests

### Atomic Test #1 - Enable Windows Remote ManagementPowershell Enable WinRM

Upon successful execution, powershell will "Enable-PSRemoting" allowing for remote PS access.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Enable-PSRemoting -Force
```

In [ ]:
Invoke-AtomicTest T1021.006 -TestNumbers 1

### Atomic Test #2 - Remote Code Execution with PS Credentials Using Invoke-CommandSimulate lateral movement with PowerShell Remoting on the local host. 
Upon successful execution, PowerShell will execute `whoami` using `Invoke-Command`, targeting the 
local machine as remote target.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Enable-PSRemoting -Force
Invoke-Command -ComputerName $env:COMPUTERNAME -ScriptBlock {whoami}```

In [ ]:
Invoke-AtomicTest T1021.006 -TestNumbers 2

#### Cleanup: 
```powershell
Disable-PSRemoting -Force```

In [ ]:
Invoke-AtomicTest T1021.006 -TestNumbers 2 -Cleanup

### Atomic Test #3 - WinRM Access with Evil-WinRM
An adversary may attempt to use Evil-WinRM with a valid account to interact with remote systems that have WinRM enabled
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must have Ruby Installed
##### Check Prereq Commands:
```powershell
try {if (ruby -v) {exit 0} else {exit 1}} catch {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest  -OutFile "PathToAtomicsFolder\..\ExternalPayloads\rubyinstaller-2.7.1-1-x64.exe" https://github.com/oneclick/rubyinstaller2/releases/download/RubyInstaller-2.7.1-1/rubyinstaller-2.7.1-1-x64.exe
$file1= "PathToAtomicsFolder\..\ExternalPayloads\rubyinstaller-2.7.1-1-x64.exe"
Start-Process $file1 /S;
```
##### Description: Computer must have Evil-WinRM installed
##### Check Prereq Commands:
```powershell
try {if (evil-winrm -h) {exit 0} else {exit 1}} catch {exit 1}
```
##### Get Prereq Commands:
```powershell
gem install evil-winrm
```

In [ ]:
Invoke-AtomicTest T1021.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
evil-winrm -i Target -u Domain\Administrator -p P@ssw0rd1```

In [ ]:
Invoke-AtomicTest T1021.006 -TestNumbers 3

## Detection
Monitor use of WinRM within an environment by tracking service execution. If it is not normally used or is disabled, then this may be an indicator of suspicious behavior.  Monitor processes created and actions taken by the WinRM process or a WinRM invoked script to correlate it with other related events.(Citation: Medium Detecting Lateral Movement) Also monitor for remote WMI connection attempts (typically over port 5985 when using HTTP and 5986 for HTTPS).